## Connection to Cohere
https://python.langchain.com/docs/integrations/chat/cohere


The Cohere API offers a free tier for testing purposes. However, the exact number of tokens that can be used with the free version depends on the specific tier and usage limits.\n\nFor the Free Tier: Trial plan, you have a limited number of tokens that you can use on a monthly basis. The exact number of tokens varies according to your usage behavior and can be viewed in the Cohere API dashboard. \n\nAdditionally, the Free Tier: Trial plan has other usage limits such as the maximum number of requests per minute, the maximum number of documents in a batch, and the maximum document size.\n\nIt's important to note that the Free Tier is intended for testing and experimentation, and may not be suitable for production-level usage. If you require more tokens or higher usage limits, you can upgrade to one of Cohere's paid plans, which offer more flexibility and higher limits. \n\nI hope this answers your question. If you would like more information on pricing and plans, please visit the Cohere website.

## Cohere embeddings
https://python.langchain.com/docs/integrations/text_embedding/cohere

## ChromaDB

Supported embeddings functions
https://docs.trychroma.com/embeddings

---
Settings for chroma db

In [8]:
import chromadb
chroma_client = chromadb.Client()
# This allows us to create a client that connects to the server
collection = chroma_client.create_collection(name="my_collection")

In Chroma, the chroma_client.create_collection/get_or_create_collection method allows us to create a collection of embedding vectors. collection is like a table in the relational database.

In [18]:
from chromadb.utils import embedding_functions

#openai_ef = embedding_functions.OpenAIEmbeddingFunction(model_name="text-embedding-ada-002")
cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key='api_key',  model_name="large")
metadata_options = {
    "hnsw:space": "ip"  # You can change this to "ip" or "cosine" if needed
}


collection = chroma_client.get_or_create_collection(
    name="my_collection", metadata=metadata_options, embedding_function=cohere_ef)

[similarity-metrics](https://medium.com/@junxie2/semantic-search-similarity-metrics-e215a0d65885)

Now, let’s proceed to store data in our collection. To begin, we’ll define a text_splitter that assists in breaking down documents into smaller chunks. Following that, we’ll generate a unique UUID for each segment and then insert both the documents and their respective UUIDs into the collection.

In [1]:
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter

content = 'file_content'

text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"], chunk_size=200, chunk_overlap=30)

docs = text_splitter.create_documents([content])


In [19]:
for doc in docs:
    uuid_name = uuid.uuid1()
    print("document for", uuid_name)
    collection.add(ids=[str(uuid_name)], documents=doc.page_content)

document for d4bb247c-ad79-11ee-85ec-cc483a655d66
document for d4f1a03f-ad79-11ee-a1c6-cc483a655d66
document for d5289bb0-ad79-11ee-b5d9-cc483a655d66
document for d55d661b-ad79-11ee-9f71-cc483a655d66
document for d59e55b7-ad79-11ee-948c-cc483a655d66


Ultimately, our objective is to **retrieve the pertinent documents from the database**. To achieve this, we employ the collection.query option. During the querying process, we will provide the input text and specify the number of results we wish to retrieve from the database.

In [52]:
question = 'quien es Zara?'

In [54]:
context = collection.query(query_texts=[question], n_results=1)['documents'][0]